In [1]:
import pyvisa as visa
import sounddevice as sd
import soundfile as sf
import numpy as np
from time import sleep
import wave as wav
from contextlib import closing
import struct

In [2]:
t_freq = 1    # Time per frequency
t_pause = 0.1  # Pause between two frequencies
n_freq = 20   # Number of different frequencies
f_min = 500    # Minimum frequency
f_max = 20e3  # Maximum frequency

f_c = 40e3    # Carrier frequency

fs = 300e3    # Samplingrate
vf = [500, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000] #np.linspace(f_min, f_max, n_freq, dtype=np.int64)

# Configure Waveform Generator

In [3]:
def enableAWG(inst, channel: int = 1):
  inst.write(f"OUTP{channel} ON")

def disableAWG(inst, channel: int = 1):
  inst.write(f"OUTP{channel} OFF")

def configureSignal(inst, f: int, amplitude: float, offset:float, function_type: str = "SIN", channel: int = 1):
  base = f"SOUR{channel}"

  inst.write(f"{base}:FUNC {function_type}; FREQ {int(f)}; VOLT {amplitude}")
  inst.write(f"{base}:VOLT:OFFS {offset}")

def enableModulation(inst, type: str, m: float, f: int, function_type: str = "SIN", channel: int = 1):
  base = f"SOUR{channel}:{type}"
  inst.write(f"{base}:STAT ON; INT:FREQ {f}")

  if type == "AM":
    inst.write(f"{base}:DEPT {int(m * 100)}")
  elif type == "FM":
    inst.write(f"{base}:DEV {int(m)}")

def disableModulation(inst, type: str, channel: int = 1):
  inst.write(f"SOUR{channel}:{type}:STAT OFF")

In [4]:
rm = visa.ResourceManager("@py")
resources = rm.list_resources()
resources

('USB0::10893::36097::CN61150030::0::INSTR',)

In [5]:
wav_gen_id = resources[0]
wav_gen = rm.open_resource(wav_gen_id)

## Set Carrier Signal

In [6]:
configureSignal(wav_gen, f_c, 3, 2.5)

# Measure

In [4]:
modulation_type="AM"
angle=0

t_record = (t_freq + t_pause + 0.5) #ca. 2s
channels = 1
fs_record = 44100

In [12]:
disableAWG(wav_gen)

rec = np.zeros(shape=(fs_record * t_record, channels), dtype=np.float32)
for f in vf:
  sd.rec(out=rec)
  sleep(t_pause)
  enableModulation(wav_gen, modulation_type, 0.8, f)
  enableAWG(wav_gen)
  sleep(t_freq)
  disableAWG(wav_gen)
  sd.stop()

  sf.write(f"records/meas_{modulation_type}_{f*1e-3}kHz_{angle}deg.wav", rec, fs_record, "PCM_24")


## Close Device

In [17]:
rm.close()

In [7]:
t_record = 5
channels = [3, 4]

rec = np.zeros(shape=(fs_record * t_record, len(channels)), dtype=np.float32)
sd.rec(out=rec, mapping=channels)
sd.wait()


In [8]:
sd.play(rec)
sd.wait()

> 0 H5, Core Audio (4 in, 2 out)
  1 BlackHole 16ch, Core Audio (16 in, 16 out)
  2 BlackHole 2ch, Core Audio (2 in, 2 out)
  3 MacBook Pro Mikrofon, Core Audio (1 in, 0 out)
< 4 MacBook Pro-Lautsprecher, Core Audio (0 in, 2 out)
  5 Microsoft Teams Audio, Core Audio (2 in, 2 out)
  6 Headphones + Virutal Audio, Core Audio (0 in, 2 out)